## Background

The experiment paradigm for the center-out task can be illustrated by the figure below.

![Experiment paradigm](./center_out_paradigm.jpg)

A brief introduction to the EMG signal acquisition is shown in the figure below.

![EMGs](./center_out_EMG.jpg)

## Loading the data file

Here I only include EMGs and necessary trial and timing information to make things simple. The codes below show how to read the data files, and also explain the meaning of each field.

In [9]:
import fnmatch, os
import numpy as np
import pickle

path = 'E:/Box Sync/Jango/'
file_list = fnmatch.filter(os.listdir(path), "*.pkl")
file_list = np.sort(file_list)

with open (path + file_list[0], 'rb') as fp:
    data = pickle.load(fp)

There are several files in this folder. They are separate experiment sessions recorded at different time in a single day or on different days. The codes above show how to load the first file in the list. After loading the file, we get a dictionary named as `data`. The fields in `data` are as follows:

In [11]:
print(data.keys())

dict_keys(['filtered_EMG', 'timeframe', 'trial_start_time', 'trial_end_time', 'trial_gocue_time', 'trial_target_dir', 'EMG_names', 'trial_result'])


## Understanding the fields in the dictionary

### EMG data, timeframe and muscle names

Here `data['filtered_EMG']` contains the EMG envelopes after 10-Hz lowpass filtering. The sampling frequency here is 1000 Hz. The columns of `data['filtered_EMG']` are muscle, with names specified by `data['EMG_names']`. The rows of `data['filtered_EMG']` are time. `data['timeframe']` contains the timestamps for each sampling point. One thing you need to notice is that the first timestamp in `data['timeframe']` is not always 0.0. 

The codes below show how to downsample the EMGs and generate the correct timeframe.

In [37]:
from scipy.signal import decimate

new_fs = 20 # Hz
# Downsampling the EMGs to 20 Hz. The downsampling factor. When using IIR downsampling (the default option in the function), 
# it is recommended to call decimate multiple times for downsampling factors higher than 13.
EMG_20Hz = np.asarray([decimate(decimate(each, 5), 10) for each in list(data['filtered_EMG'].T)]).T

# Get the new timestamps for the downsampled
t = data['timeframe'][0] + np.arange(0, EMG_20Hz.shape[0])/new_fs

Picking EMG channels according to names. First let's print out all the EMG names

In [42]:
print(data['EMG_names'])

['EMG_FCRr', 'EMG_FCUu', 'EMG_FDPr', 'EMG_FDPu', 'EMG_FDSu', 'EMG_PL', 'EMG_PT', 'EMG_APB', 'EMG_FPB', 'EMG_FDS', 'EMG_ECU', 'EMG_ECRb', 'EMG_EDC1', 'EMG_Brad']


Here I recommend to start with the 4 wrist muscles, two flexors and two extensors: FCRr, FCUu, ECU, ECRb. The codes below help you to get their indices and pick them up. Here I use the downsampled EMG as an example.

In [55]:
EMG_list = ['EMG_FCRr', 'EMG_FCUu', 'EMG_ECU', 'EMG_ECRb'] # Here list the EMG names you want to include in your analysis
ind = [data['EMG_names'].index(each) for each in EMG_list]
picked_EMG = EMG_20Hz[:, ind]
print('Here we picked %d channels, they are %s'%(picked_EMG.shape[1], EMG_list))

Here we picked 4 channels, they are ['EMG_FCRr', 'EMG_FCUu', 'EMG_ECU', 'EMG_ECRb']


### Trial information and behavior events

These information includes:
* `data['trial_result']`: the results of the trials. 'R' indicates the trial was successful and the monkey got reward. 'A' means an aborted trial. 'F' means a failed trial. In those file most trials (>98%) are successful. So no need to bother the differences between rewarded trials and failed trials
* `data['trial_start_time']`, `data['trial_end_time']`, `data['trial_gocue_time']`: self-explanatory names for the timings of the behavior events, see the first figure above. The unit is seconds. Data between each `data['trial_end_time']` and the next `data['trial_start_time']` are the inter-trial data. Mostly the monkey's wrist remained still during those inter-trial periods.
* `data['trial_target_dir']` gives the target direction for each trial, also see the first figure above. They can be viewed as the class of each state. The 'inter-trial' data have no such 'target directions', but they can be viewed as an additional class.

## Test the algorithms

To evaluate the accuracy of segmentation, we can compare the true timings in `data['trial_start_time']`, `data['trial_end_time']` with the state transition timings obtained by the algorithm.

Meanwhile, to test whether the algorithm assigns the right class (state) lable to each segment, we need to compare the inferred class labels, namely the inferred target directions, with the true target directions. For the 'inter-trial' data, they can be viewed as an 'inter-trial' class.